In [1]:
%load_ext autoreload
%autoreload complete --print

In [2]:
from pathlib import Path

(Path(".") / "my_db.db").unlink(missing_ok=True)

In [3]:
from src import Session, User, Base
from wonderwords import RandomWord
import os

In [4]:
# Generates random words/sentences, useful for testing.
ranword = RandomWord()

In [5]:
# Create all tables.
with Session.begin() as sess:
    Base.metadata.create_all(sess.bind)

2024-09-08 20:38:32,470 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-09-08 20:38:32,470 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("user")
2024-09-08 20:38:32,471 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-09-08 20:38:32,472 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("user")
2024-09-08 20:38:32,473 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-09-08 20:38:32,474 INFO sqlalchemy.engine.Engine 
CREATE TABLE user (
	id INTEGER NOT NULL, 
	created DATETIME NOT NULL, 
	username VARCHAR NOT NULL, 
	passwd VARCHAR NOT NULL, 
	is_active BOOLEAN NOT NULL, 
	nickname VARCHAR, 
	PRIMARY KEY (id)
)


2024-09-08 20:38:32,475 INFO sqlalchemy.engine.Engine [no key 0.00052s] ()
2024-09-08 20:38:32,490 INFO sqlalchemy.engine.Engine CREATE UNIQUE INDEX ix_user_username ON user (username)
2024-09-08 20:38:32,491 INFO sqlalchemy.engine.Engine [no key 0.00086s] ()
2024-09-08 20:38:32,497 INFO sqlalchemy.engine.Engine COMMIT


In [11]:
with Session() as sess:
    user = User(username=ranword.word(), passwd=os.urandom(15).hex())
    sess.add(user)
    sess.commit()
    print(user)

with Session.begin() as sess:
    user = User(username=ranword.word(), passwd=os.urandom(15).hex())
    sess.add(user)
    print(user)

# TODO: Make the whole echo-ing thing easy to turn on/off for learning purposes.
# NOTE: Every read of user is query-backed, which means automatic properties like
# defaults or autoincrementing ids aren't visible till commit.
# In a session.begin() context, you cannot explicitly commit, so reading automatic
# properties within the same context will return None for any automatic property,
# even if said property is non-nullable or is supposed to have a default value.

2024-09-08 20:43:14,657 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-09-08 20:43:14,658 INFO sqlalchemy.engine.Engine INSERT INTO user (created, username, passwd, is_active, nickname) VALUES (CURRENT_TIMESTAMP, ?, ?, ?, ?) RETURNING id, created
2024-09-08 20:43:14,659 INFO sqlalchemy.engine.Engine [cached since 280.2s ago] ('shack', '682294e49b64580d7022069739ab3b', 1, None)
2024-09-08 20:43:14,659 INFO sqlalchemy.engine.Engine COMMIT
2024-09-08 20:43:14,673 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-09-08 20:43:14,674 INFO sqlalchemy.engine.Engine SELECT user.id AS user_id, user.created AS user_created, user.username AS user_username, user.passwd AS user_passwd, user.is_active AS user_is_active, user.nickname AS user_nickname 
FROM user 
WHERE user.id = ?
2024-09-08 20:43:14,675 INFO sqlalchemy.engine.Engine [cached since 218.7s ago] (6,)
User(id=6, username='shack', created=datetime.datetime(2024, 9, 8, 12, 43, 14), is_active=True, nickname=None)
2024-09-08 20:43:14,6